In [1]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

### Load excel file with table and derivative variables of interest

In [2]:
# Input csv or excel filename & filepath
fname = 'abcd_mid_tfmri_pull.xlsx'
fpath = '/Users/cglab/data_pull/abcd/'
#######################################
if fname.endswith('xlsx'):
    tvars = pd.read_excel(fpath + fname)
elif fname.endswith('csv'):
    tvars = pd.read_csv(fpath + fname)
else:
    print('unexpected filetype')
tvars.tail()

,Table,Variable,Description,Unnamed: 3,Unnamed: 4,Unnamed: 5
42,abcd_p_demo,race_ethnicity,NaN,NaN,NaN,NaN
43,abcd_p_demo,acs_raked_propensity_score,NaN,NaN,NaN,NaN
44,led_l_adi,reshist_addr1_adi_edu_l,Residential history derived - Area Deprivation...,NaN,NaN,NaN
45,led_l_adi,reshist_addr1_adi_sp,Residential history derived - Area Deprivation...,NaN,NaN,NaN
46,led_l_adi,reshist_addr1_adi_unemp,Residential history derived - Area Deprivation...,NaN,NaN,NaN


In [3]:
dpath = '/home/cglab/projects/abcd/data/abcd5.1-rser/'

### Functions

In [4]:
# recursively search directory and subdirectories for csv file
def findFile(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
            
def getDerivatives(df, table_file, table_key, how_merge='outer'):
    # set file path for raw data
    dpath = '/Users/cglab/projects/abcd/data/abcd-data-release-5.1/'
    # find filepath, which is full path and name of file
    filepath = findFile(table_file, path=dpath)
    dat = pd.read_csv(filepath, low_memory=False)
    # get table name, which is the string before the period
    table = table_file.split('.')[0]
    # get column names sub and event which will need for merging dataframes
    # it's the same for each df so overwriting is fine
    se_nms = dat.columns[:2].values.tolist()
    # derivative variables desired
    derivative_cols = table_key[table_key['Table']==table]['Variable'].values.tolist()
    derivative_cols += se_nms
    # strip leading and ending spaces
    derivative_cols = [c.strip() for c in derivative_cols]
    # # troubleshooting
    # print(derivative_cols)
    # print(df.columns)
    # merge with overall with INNER join bc we dont want to exclude participants who have task mri data but not resting or vice versa
    print('Prior to merge df size is {0} and other df shape is {1}\n merging...'.format(df.shape, dat[derivative_cols].shape))
    if 'src_subject_id' not in df.columns:
        # for first table assign it to df
        df = dat[derivative_cols].copy()
    else:
        # all others are merged
        df = df.merge(dat[derivative_cols], how=how_merge, on=['src_subject_id', 'eventname'])
    print('Any duplicated columns? {}'.format(df.columns.duplicated().any()))
    print('New df size is {}\n'.format(df.shape))
    return df


def dropDuplicateCols(df):
    # check every combination of columns
    # if ALL values are EQUAL, get the 2 columns
    dup_pairs = [(i, j) for i,j in combinations(df, 2) if df[i].equals(df[j])]
    cols_to_remove = []
    # print the columns that match and get 1st one for removal
    for i, j in dup_pairs:
        # check if alll rows in matching column are NaN or 0. if so, don't delete, because this function is
        # for identifying columns with identical values. dropping fully missing variables should be done separetly for clarity
        if ~df[i].isnull().all() and rs.loc[rs[i].notnull(), i].sum() > 0:
            print('{0} is identical to {1}'.format(i.upper(), j.upper()))
            cols_to_remove.append(i)    
    # drop duplicate columns
    df.drop(columns=cols_to_remove, inplace=True)
    # return df WITHOUT duplicate columns
    return df

#check if there are any MRI variables requested
# if so, return the appropriate Quality control Variables
def anyMRIvars(df):
    # create empty list to store QC vars needed
    qc_vars = []

    # check for RESTING variables
    # if any columns are found that startwith 'rsfmri_', append RESTING QC vars & the csv file they're found in,..
    # ... creating a list of tuples which will be combined and converted to dataframe below
    # this relies on the implicit boolean nature of lists, if empty their FALSE
    if [col for col in df.columns if col.startswith('rsfmri_')]:
        qc_vars += [('mri_y_qc_incl', 'imgincl_rsfmri_include'), ('mri_y_qc_motion', 'rsfmri_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]
    
    # check for NBACK task variables
    # if any columns are found that startwith 'tfmri_nback', append NBACK QC vars & the csv file they're found in
    if [col for col in df.columns if col.startswith('tfmri_nback_') or col.startswith('tfabwdp')  or col.startswith('tnbasem')]:
        qc_vars += [('mri_y_qc_incl', 'imgincl_nback_include'), ('mri_y_qc_motion', 'tfmri_nback_all_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]
    
    # check for MID task variables
    # if any columns are found that startwith 'tfmri_ma', append MID QC vars & the csv file they're found in
    if [col for col in df.columns if col.startswith('tfmri_ma_') or col.startswith('midabwdp')]:
        qc_vars += [('mri_y_qc_incl', 'imgincl_mid_include'), ('mri_y_qc_motion', 'tfmri_mid_all_meanmotion'), ('mri_y_adm_info', 'mri_info_deviceserialnumber')]
    
    # if qc_vars is NOT empty
    if qc_vars:
        # remove duplicates from qc_vars
        # calling 'set' function on qc_vars returns only unique items
        # then calling 'list' function converts it back to a list
        qc_vars = list(set(qc_vars))
        # set up table_key dataframe for pulling QC vars
        # includes all QC vars selected above with cooresponding table name
        qc_key = pd.DataFrame(qc_vars, columns=['Table', 'Variable'])
        # get list of unique QC tables from qc_key tuple ex. ('mri_y_qc_incl', 'imgincl_rsfmri_include')
        qc_tables = list(set([table for table, var in qc_vars]))
        # append QC variables to dataframe
        for table in qc_tables:
            print('\nPulling from table: {0}.csv\n'.format(table))
            df = getDerivatives(df, table + '.csv', qc_key, how_merge='left')
        # output dataframe with QC variables merged
        return df
    else:
        # else return the unchanged dataframe
        return df
        

### Get tables that will be pulled
* append '.csv' for data loading later
* exclude temporal variance tables because interpretation of them unclear

In [5]:
[t for t in tvars['Table'].unique()]

['mri_y_tfmr_mid_allvn_aseg',
 'mri_y_tfmr_mid_alrvn_aseg',
 'mri_y_tfmr_mid_allvn_dsk',
 'mri_y_tfmr_mid_alrvn_dsk',
 'mri_y_tfmr_mid_allvn_dst',
 'mri_y_tfmr_mid_alrvn_dst',
 'abcd_y_lt',
 'abcd_p_demo',
 'led_l_adi']

In [6]:
data_files = [t + '.csv' for t in tvars['Table'].unique()]
data_files

['mri_y_tfmr_mid_allvn_aseg.csv',
 'mri_y_tfmr_mid_alrvn_aseg.csv',
 'mri_y_tfmr_mid_allvn_dsk.csv',
 'mri_y_tfmr_mid_alrvn_dsk.csv',
 'mri_y_tfmr_mid_allvn_dst.csv',
 'mri_y_tfmr_mid_alrvn_dst.csv',
 'abcd_y_lt.csv',
 'abcd_p_demo.csv',
 'led_l_adi.csv']

## Make sure that first csv that is pulled from has multiple 'eventnames', so that they're included in future merges

### Load, Select, & Merge data, 
* for every file in 'data_files'
* and get every variable listed in 'tvars'
* append to rs

In [7]:
# set empty dataframe  
# to be filled with merges
rs = pd.DataFrame()

for file in data_files:
    rs = getDerivatives(rs, file, tvars)

Prior to merge df size is (0, 0) and other df shape is (20279, 10)
 merging...
Any duplicated columns? False
New df size is (20279, 10)

Prior to merge df size is (20279, 10) and other df shape is (20283, 10)
 merging...
Any duplicated columns? False
New df size is (20284, 18)

Prior to merge df size is (20284, 18) and other df shape is (20279, 10)
 merging...
Any duplicated columns? False
New df size is (20284, 26)

Prior to merge df size is (20284, 26) and other df shape is (20283, 10)
 merging...
Any duplicated columns? False
New df size is (20284, 34)

Prior to merge df size is (20284, 34) and other df shape is (20279, 4)
 merging...
Any duplicated columns? False
New df size is (20284, 36)

Prior to merge df size is (20284, 36) and other df shape is (20283, 4)
 merging...
Any duplicated columns? False
New df size is (20284, 38)

Prior to merge df size is (20284, 38) and other df shape is (90312, 4)
 merging...
Any duplicated columns? False
New df size is (90312, 40)

Prior to merge

In [8]:
idx = ['site_id_l', 'interview_age', 'rel_family_id', 'src_subject_id', 'eventname', 'demo_sex_v2', 'acs_raked_propensity_score', 'race_ethnicity', 'demo_prnt_marital_v2', 'demo_prnt_ed_v2', 'demo_prtnr_ed_v2', 'demo_comb_income_v2', 'demo_comb_income_v2_l', 'latent_factor_ss_general_ses', 'latent_factor_ss_social', 'latent_factor_ss_perinatal', 'aeq_positive_expectancies_ss', 'aeq_negative_expectancies_ss', 'aeq_negative_expectancies_nt', 'aeq_positive_expectancies_nt', 'aeq_section_q01',
       'aeq_section_q02', 'aeq_section_q03', 'aeq_section_q04', 'aeq_section_q05', 'aeq_section_q06', 'aeq_section_q07']

In [9]:
# show any duplicated columns
rs.loc[:, rs.columns.duplicated()==1]

""
0
1
2
3
4
...
90307
90308
90309
90310


### Drop columns with duplicate NAMES

In [10]:
rs = rs.loc[:,~rs.columns.duplicated()]
print('Any duplicated columns? {}'.format(rs.columns.duplicated().any()))

Any duplicated columns? False


In [11]:
rs.head()

,tfmri_ma_aclvn_b_scs_aalh,tfmri_ma_aclvn_b_scs_aarh,tfmri_ma_aclvn_b_scs_aylh,tfmri_ma_aclvn_b_scs_ayrh,tfmri_ma_aclvn_b_scs_hpuslh,tfmri_ma_aclvn_b_scs_hpusrh,tfmri_ma_aclvn_b_scs_tplh,tfmri_ma_aclvn_b_scs_tprh,src_subject_id,eventname,tfmri_ma_alrvn_b_scs_aalh,tfmri_ma_alrvn_b_scs_aarh,tfmri_ma_alrvn_b_scs_aylh,tfmri_ma_alrvn_b_scs_ayrh,tfmri_ma_alrvn_b_scs_hpuslh,tfmri_ma_alrvn_b_scs_hpusrh,tfmri_ma_alrvn_b_scs_tplh,tfmri_ma_alrvn_b_scs_tprh,tfmri_ma_allvn_b_cds_lobofrlh,tfmri_ma_allvn_b_cds_lobofrrh,tfmri_ma_allvn_b_cds_mobofrlh,tfmri_ma_allvn_b_cds_mobofrrh,tfmri_ma_allvn_b_cds_roatcgelh,tfmri_ma_allvn_b_cds_roatcgerh,tfmri_ma_allvn_b_cds_clatcgelh,tfmri_ma_allvn_b_cds_clatcgerh,tfmri_ma_alrvn_b_cds_clatcgelh,tfmri_ma_alrvn_b_cds_clatcgerh,tfmri_ma_alrvn_b_cds_lobofrlh,tfmri_ma_alrvn_b_cds_lobofrrh,tfmri_ma_alrvn_b_cds_mobofrlh,tfmri_ma_alrvn_b_cds_mobofrrh,tfmri_ma_alrvn_b_cds_roatcgelh,tfmri_ma_alrvn_b_cds_roatcgerh,midabwdp1083,midabwdp1157,midabwdp639,midabwdp713,rel_family_id,interview_age,demo_comb_income_v2,demo_prnt_ed_v2,demo_prtnr_ed_v2,demo_gender_id_v2,race_ethnicity,acs_raked_propensity_score,reshist_addr1_adi_edu_l,reshist_addr1_adi_sp,reshist_addr1_adi_unemp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NDAR_INV003RTV85,18_month_follow_up_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143.0,NaN,NaN,NaN,NaN,1.0,533.38182,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NDAR_INV003RTV85,30_month_follow_up_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop columns with duplicate VALUES

In [12]:
print(rs.shape)
rs = dropDuplicateCols(rs)
print(rs.shape)

(90312, 49)
(90312, 49)


In [13]:
if 'rsfmri_c_ngd_dt_ngd_dt' in rs.columns.tolist():
    print('included')

In [14]:
# check number of missing values in each column
rs.isnull().sum()

tfmri_ma_aclvn_b_scs_aalh         70037
tfmri_ma_aclvn_b_scs_aarh         70037
tfmri_ma_aclvn_b_scs_aylh         70037
tfmri_ma_aclvn_b_scs_ayrh         70037
tfmri_ma_aclvn_b_scs_hpuslh       70037
tfmri_ma_aclvn_b_scs_hpusrh       70037
tfmri_ma_aclvn_b_scs_tplh         70037
tfmri_ma_aclvn_b_scs_tprh         70037
src_subject_id                        0
eventname                             0
tfmri_ma_alrvn_b_scs_aalh         70033
tfmri_ma_alrvn_b_scs_aarh         70033
tfmri_ma_alrvn_b_scs_aylh         70033
tfmri_ma_alrvn_b_scs_ayrh         70033
tfmri_ma_alrvn_b_scs_hpuslh       70033
tfmri_ma_alrvn_b_scs_hpusrh       70033
tfmri_ma_alrvn_b_scs_tplh         70033
tfmri_ma_alrvn_b_scs_tprh         70033
tfmri_ma_allvn_b_cds_lobofrlh     70037
tfmri_ma_allvn_b_cds_lobofrrh     70037
tfmri_ma_allvn_b_cds_mobofrlh     70037
tfmri_ma_allvn_b_cds_mobofrrh     70037
tfmri_ma_allvn_b_cds_roatcgelh    70037
tfmri_ma_allvn_b_cds_roatcgerh    70037
tfmri_ma_allvn_b_cds_clatcgelh    70037


In [15]:
rs.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
6_month_follow_up_arm_1     11389
1_year_follow_up_y_arm_1    11220
18_month_follow_up_arm_1    11083
2_year_follow_up_y_arm_1    10973
3_year_follow_up_y_arm_1    10336
30_month_follow_up_arm_1    10232
42_month_follow_up_arm_1     8457
4_year_follow_up_y_arm_1     4754
Name: count, dtype: int64

### Check if any MRI variables present
* if so, append the appropriate ABCD Quality Control Variables
* for scanner movement, artifacts, etc

In [16]:
print(rs.shape)
rs = anyMRIvars(rs)
print(rs.shape)

(90312, 49)

Pulling from table: mri_y_qc_incl.csv

Prior to merge df size is (90312, 49) and other df shape is (22939, 3)
 merging...
Any duplicated columns? False
New df size is (90312, 50)


Pulling from table: mri_y_qc_motion.csv

Prior to merge df size is (90312, 50) and other df shape is (22460, 3)
 merging...
Any duplicated columns? False
New df size is (90312, 51)


Pulling from table: mri_y_adm_info.csv

Prior to merge df size is (90312, 51) and other df shape is (22939, 3)
 merging...
Any duplicated columns? False
New df size is (90312, 52)

(90312, 52)


In [17]:
rs['eventname'].value_counts()

eventname
baseline_year_1_arm_1       11868
6_month_follow_up_arm_1     11389
1_year_follow_up_y_arm_1    11220
18_month_follow_up_arm_1    11083
2_year_follow_up_y_arm_1    10973
3_year_follow_up_y_arm_1    10336
30_month_follow_up_arm_1    10232
42_month_follow_up_arm_1     8457
4_year_follow_up_y_arm_1     4754
Name: count, dtype: int64

In [18]:
rs.describe()

,tfmri_ma_aclvn_b_scs_aalh,tfmri_ma_aclvn_b_scs_aarh,tfmri_ma_aclvn_b_scs_aylh,tfmri_ma_aclvn_b_scs_ayrh,tfmri_ma_aclvn_b_scs_hpuslh,tfmri_ma_aclvn_b_scs_hpusrh,tfmri_ma_aclvn_b_scs_tplh,tfmri_ma_aclvn_b_scs_tprh,tfmri_ma_alrvn_b_scs_aalh,tfmri_ma_alrvn_b_scs_aarh,tfmri_ma_alrvn_b_scs_aylh,tfmri_ma_alrvn_b_scs_ayrh,tfmri_ma_alrvn_b_scs_hpuslh,tfmri_ma_alrvn_b_scs_hpusrh,tfmri_ma_alrvn_b_scs_tplh,tfmri_ma_alrvn_b_scs_tprh,tfmri_ma_allvn_b_cds_lobofrlh,tfmri_ma_allvn_b_cds_lobofrrh,tfmri_ma_allvn_b_cds_mobofrlh,tfmri_ma_allvn_b_cds_mobofrrh,tfmri_ma_allvn_b_cds_roatcgelh,tfmri_ma_allvn_b_cds_roatcgerh,tfmri_ma_allvn_b_cds_clatcgelh,tfmri_ma_allvn_b_cds_clatcgerh,tfmri_ma_alrvn_b_cds_clatcgelh,tfmri_ma_alrvn_b_cds_clatcgerh,tfmri_ma_alrvn_b_cds_lobofrlh,tfmri_ma_alrvn_b_cds_lobofrrh,tfmri_ma_alrvn_b_cds_mobofrlh,tfmri_ma_alrvn_b_cds_mobofrrh,tfmri_ma_alrvn_b_cds_roatcgelh,tfmri_ma_alrvn_b_cds_roatcgerh,midabwdp1083,midabwdp1157,midabwdp639,midabwdp713,rel_family_id,interview_age,demo_comb_income_v2,demo_prnt_ed_v2,demo_prtnr_ed_v2,demo_gender_id_v2,race_ethnicity,acs_raked_propensity_score,reshist_addr1_adi_edu_l,reshist_addr1_adi_sp,reshist_addr1_adi_unemp,imgincl_mid_include,tfmri_mid_all_meanmotion
count,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20275.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20275.000000,20275.000000,20279.000000,20279.000000,11868.000000,90304.000000,11866.000000,11868.000000,9470.000000,11866.000000,23079.000000,23088.000000,11214.000000,11214.000000,11214.000000,22939.000000,20355.000000
mean,0.007521,0.012343,-0.043977,-0.038310,-0.024163,-0.014804,0.025946,0.031292,0.091595,0.086542,-0.022966,-0.019337,-0.005089,0.000456,0.040265,0.049354,-0.009632,0.004434,-0.041573,-0.043429,-0.033407,-0.019839,0.018782,0.025805,0.045537,0.053906,0.002891,0.023700,-0.018507,-0.022219,0.016899,0.018283,0.040879,0.063842,0.054980,0.091664,5955.899141,140.532258,82.498904,17.684193,22.917318,2.200573,2.031674,710.340651,4.951707,18.079994,9.067018,0.810628,0.277899
std,0.361885,0.369423,0.300532,0.286976,0.208527,0.198800,0.190365,0.198432,0.378899,0.376121,0.294781,0.287361,0.212981,0.201631,0.198908,0.215446,0.357839,0.368314,0.535252,0.513389,0.281906,0.278458,0.234950,0.229447,0.247143,0.238401,0.370690,0.384369,0.546488,0.517029,0.286877,0.284856,0.259067,0.260001,0.266995,0.271287,3430.136718,16.714658,248.275498,28.892921,78.518592,26.232962,1.325221,440.937045,6.906475,12.915207,6.052322,0.391812,0.328756
min,-6.732612,-8.083790,-7.946914,-4.573852,-3.702281,-2.781994,-2.345739,-2.116597,-7.342105,-5.988147,-5.466525,-5.583201,-5.679289,-4.891436,-3.354646,-3.745621,-6.560557,-7.517347,-7.649204,-7.934921,-4.977736,-3.906199,-2.349866,-2.373990,-6.829726,-5.940623,-6.510240,-7.664679,-8.121262,-10.169366,-6.446625,-7.744891,-5.411687,-3.624256,-5.202297,-5.155500,0.000000,107.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021262
25%,-0.147838,-0.143891,-0.174560,-0.163700,-0.126239,-0.114508,-0.071316,-0.070016,-0.071008,-0.074719,-0.152637,-0.142853,-0.109686,-0.100785,-0.062688,-0.055879,-0.175708,-0.166989,-0.288186,-0.270016,-0.169578,-0.152742,-0.104986,-0.092284,-0.081787,-0.068299,-0.163473,-0.151898,-0.276534,-0.257730,-0.126957,-0.118045,-0.090023,-0.070919,-0.080586,-0.049720,2977.750000,128.000000,6.000000,15.000000,15.000000,1.000000,1.000000,421.841839,0.974589,8.936171,5.039299,1.000000,0.105178
50%,0.014745,0.016347,-0.041659,-0.036616,-0.021649,-0.012325,0.028193,0.035210,0.100467,0.095062,-0.021312,-0.014839,-0.002274,0.004199,0.044124,0.051974,-0.007552,0.008283,-0.037388,-0.042423,-0.027310,-0.015075,0.022592,0.029539,0.0497

### DONT run, unless Only MR waves are desired
### Waves with MRI data to retain

In [19]:
waves = ['baseline_year_1_arm_1', '2_year_follow_up_y_arm_1', '4_year_follow_up_y_arm_1']
rs = rs[rs['eventname'].isin(waves)]
print(rs.shape)
rs['eventname'].value_counts()

(27595, 52)


eventname
baseline_year_1_arm_1       11868
2_year_follow_up_y_arm_1    10973
4_year_follow_up_y_arm_1     4754
Name: count, dtype: int64

### Check missing in Family ID

In [20]:
rs[rs['eventname']=='baseline_year_1_arm_1']['rel_family_id'].isnull().sum(), rs['rel_family_id'].isnull().sum()

(0, 15727)

#### 0 baseline participants are missing family id
* so we can assign family id to other waves if needed

### Convert scanID string to 32 unique float numbers

In [21]:
rs['mri_info_deviceserialnumber'].nunique()

32

In [22]:
unique_vals = rs['mri_info_deviceserialnumber'].unique()
rs['mri_info_deviceserialnumber'] = rs['mri_info_deviceserialnumber'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))

rs['mri_info_deviceserialnumber'].dtype

/var/folders/g2/cs_63cw929v86s87fmtjn5lr0000gp/T/ipykernel_75097/3663062225.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rs['mri_info_deviceserialnumber'] = rs['mri_info_deviceserialnumber'].replace(to_replace=unique_vals,


dtype('int64')

### Include participants who passed Qaulity Control in ABCD preprocessing
* 'imgincl_rsfmri_include' & 'imgincl_nback_include' are provided by ABCD
* after preprocessing, raters assessed images and those receiving a 0 in these variables,
* had extremely noisey, unrecoverable data
* 1- data to include; 0- unusable
* 0 in either warrants exclusion

In [23]:
print(rs.shape)
# edit which 'imgincl_..._inlcude' to use e.g., 'imgincl_mid_include'
# keep desired non-MR waves OR MR waves where 'imgincl_mid_include' == 1
mr_waves = ['baseline_year_1_arm_1', '2_year_follow_up_y_arm_1', '4_year_follow_up_y_arm_1']
rs = rs[(~rs['eventname'].isin(mr_waves)) | (rs['eventname'].isin(mr_waves) & rs['imgincl_mid_include']==1)]
rs.shape

(27595, 52)


(18595, 52)

In [24]:
rs.loc[rs['eventname'].isin(mr_waves), 'eventname'].value_counts().sum()

18595

In [25]:
rs['eventname'].value_counts()

eventname
baseline_year_1_arm_1       9175
2_year_follow_up_y_arm_1    6744
4_year_follow_up_y_arm_1    2676
Name: count, dtype: int64

### Rename Connectivty columns
* Replace ngd with '_' in rsFC variables
* for resting state fMRI

In [26]:
# # get all column names that have 'ngd' in the name
# ngd = [c for c in rs.columns if 'ngd' in c or 'scs' in c]

# if ngd:
#     # split on '_", get the first and 3rd strings,
#     # join them together with '_' in between
#     cor = ['_'.join([c.split('_')[0], c.split('_')[2]]) for c in ngd]
#     # zip them together in a dictionary for renaming in pandas
#     ncdict = dict(zip(ngd, cor))
#     # rename in pandas
#     rs.rename(ncdict, axis=1, inplace=True)

### If they provided new names for variables

In [27]:
tvars.columns

Index(['Table', 'Variable', 'Description', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], dtype='object')

In [28]:
if 'NEWNAME' in tvars.columns:
    # create dictionary that maps the new names to original varialble names
    req_redict = dict(zip(tvars['Variable'], tvars['NEWNAME']))
    req_redict
else:
    req_redict = []

In [29]:
redict = {
    'src_subject_id': 'subID',
    'rel_family_id': 'famID',
    'interview_age': 'age',
    'race_ethnicity': 'race',
    'mri_info_deviceserialnumber': 'scanID',
    'site_id_l': 'siteID',
    'rsfmri_meanmotion': 'Motrs',
    'tfmri_mid_all_meanmotion': 'Motm',
    'tfmri_nback_all_meanmotion': 'Motnb',
    'demo_comb_income_v2': 'income',
    'demo_sex_v2': 'sex',
    'demo_prnt_ed_v2': 'pedu', 
    'demo_prtnr_ed_v2': 'spedu', 
    'demo_gender_id_v2': 'gender',
    'acs_raked_propensity_score': 'prpensity',
    'reshist_addr1_adi_edu_l': 'LowEdu1',
    'reshist_addr1_adi_sp': 'SingPH1',
    'reshist_addr1_adi_unemp': 'UnempR1',         

    # Large Loss Vs Neutral
    'tfmri_ma_aclvn_b_scs_aalh': 'NAL_lln',
    'tfmri_ma_aclvn_b_scs_aarh': 'NAR_lln',
    'tfmri_ma_aclvn_b_scs_aylh': 'AmygL_lln',
    'tfmri_ma_aclvn_b_scs_ayrh': 'AmygR_lln',
    'tfmri_ma_aclvn_b_scs_hpuslh': 'HipcL_lln',
    'tfmri_ma_aclvn_b_scs_hpusrh': 'HipcR_lln',
    'tfmri_ma_aclvn_b_scs_tplh': 'ThalL_lln',
    'tfmri_ma_aclvn_b_scs_tprh': 'ThalR_lln',
    'tfmri_ma_allvn_b_cds_lobofrlh': 'LtOrFrL_lln',
    'tfmri_ma_allvn_b_cds_lobofrrh': 'LtOrFrR_lln',
    'tfmri_ma_allvn_b_cds_mobofrlh': 'MedOrFrL_lln',
    'tfmri_ma_allvn_b_cds_mobofrrh': 'MedOrFrR_lln',
    'tfmri_ma_allvn_b_cds_roatcgelh': 'rACCL_lln',
    'tfmri_ma_allvn_b_cds_roatcgerh': 'rACCR_lln',
    'tfmri_ma_allvn_b_cds_clatcgelh': 'cACCL_lln',
    'tfmri_ma_allvn_b_cds_clatcgerh': 'cACCR_lln',
    'midabwdp1083': 'aInslL_lln',
    'midabwdp1157': 'aInslR_lln',

    # Large Reward Vs Neutral
    'tfmri_ma_alrvn_b_scs_aalh': 'NAL_lrn',
    'tfmri_ma_alrvn_b_scs_aarh': 'NAR_lrn',
    'tfmri_ma_alrvn_b_scs_aylh': 'AmygL_lrn',
    'tfmri_ma_alrvn_b_scs_ayrh': 'AmygR_lrn',
    'tfmri_ma_alrvn_b_scs_hpuslh': 'HipcL_lrn',
    'tfmri_ma_alrvn_b_scs_hpusrh': 'HipcR_lrn',
    'tfmri_ma_alrvn_b_scs_tplh': 'ThalL_lrn',
    'tfmri_ma_alrvn_b_scs_tprh': 'ThalR_lrn',    
    'tfmri_ma_alrvn_b_cds_clatcgelh': 'cACCL_lrn',
    'tfmri_ma_alrvn_b_cds_clatcgerh': 'cACCR_lrn',
    'tfmri_ma_alrvn_b_cds_lobofrlh': 'LtOrFrL_lrn',
    'tfmri_ma_alrvn_b_cds_lobofrrh': 'LtOrFrR_lrn',
    'tfmri_ma_alrvn_b_cds_mobofrlh': 'MedOrFrL_lrn',
    'tfmri_ma_alrvn_b_cds_mobofrrh': 'MedOrFrR_lrn',
    'tfmri_ma_alrvn_b_cds_roatcgelh': 'rACCL_lrn',
    'tfmri_ma_alrvn_b_cds_roatcgerh': 'rACCR_lrn',
    'midabwdp639': 'aInslL_lrn',
    'midabwdp713': 'aInslR_lrn',
}
if req_redict:
    req_redict['src_subject_id'] = 'subid'
    req_redict['mri_info_deviceserialnumber'] = 'scanid'
    rs = rs.rename(columns=req_redict)
else:
    rs = rs.rename(columns=redict)

In [30]:
rs.head()

,NAL_lln,NAR_lln,AmygL_lln,AmygR_lln,HipcL_lln,HipcR_lln,ThalL_lln,ThalR_lln,subID,eventname,NAL_lrn,NAR_lrn,AmygL_lrn,AmygR_lrn,HipcL_lrn,HipcR_lrn,ThalL_lrn,ThalR_lrn,LtOrFrL_lln,LtOrFrR_lln,MedOrFrL_lln,MedOrFrR_lln,rACCL_lln,rACCR_lln,cACCL_lln,cACCR_lln,cACCL_lrn,cACCR_lrn,LtOrFrL_lrn,LtOrFrR_lrn,MedOrFrL_lrn,MedOrFrR_lrn,rACCL_lrn,rACCR_lrn,aInslL_lln,aInslR_lln,aInslL_lrn,aInslR_lrn,famID,age,income,pedu,spedu,gender,race,prpensity,LowEdu1,SingPH1,UnempR1,imgincl_mid_include,Motm,scanID
6,0.012925,0.057350,-0.103007,0.040944,-0.278594,-0.136520,-0.226935,-0.314408,NDAR_INV003RTV85,baseline_year_1_arm_1,-0.451426,-0.238997,-0.327643,-0.223388,-0.471753,-0.240092,-0.138320,-0.261594,-0.071183,-0.014068,0.184538,0.029904,0.053321,0.144795,-0.192958,0.058817,-0.089997,-0.015292,-0.320943,-0.315482,-0.250562,-0.804303,-0.311717,-0.233716,-0.224738,-0.105134,-0.274432,-0.184748,8781.0,131.0,8.0,13.0,13.0,2.0,1.0,466.092707,NaN,NaN,NaN,1.0,0.158333,1
14,0.429318,0.272447,0.354449,0.593416,0.234241,0.289909,0.017087,0.140608,NDAR_INV005V6D2C,baseline_year_1_arm_1,0.142344,0.284221,0.240394,0.450072,0.131028,0.231011,0.043190,0.068502,0.441208,0.173635,0.620966,0.564494,0.540422,0.546842,0.132752,0.293214,-0.080159,0.038292,0.384632,0.078511,0.239707,0.132052,0.406179,0.246562,0.212062,0.198078,0.052116,-0.058588,10210.0,121.0,999.0,6.0,999.0,2.0,3.0,520.488325,4.861931,29.079160,9.991899,1.0,0.352267,2
20,-0.007370,-0.099530,-0.245714,-0.077979,-0.154156,-0.118741,-0.094763,-0.067210,NDAR_INV007W6H7B,baseline_year_1_arm_1,0.024064,-0.018880,-0.231686,-0.010857,-0.064591,-0.160605,-0.088715,-0.187258,-0.216364,-0.361254,0.003707,-0.123096,-0.181924,-0.007465,-0.092264,-0.091089,-0.186848,-0.159265,-0.259863,-0.227784,-0.275979,-0.273947,-0.375694,-0.290470,-0.060283,-0.222956,-0.010167,-0.118855,4722.0,126.0,10.0,19.0,18.0,1.0,1.0,479.185338,3.559711,0.000000,6.254295,1.0,0.086835,3
26,-0.177538,-0.140450,-0.357152,-0.321334,-0.084301,-0.276284,0.178833,0.292628,NDAR_INV00BD7VDC,baseline_year_1_arm_1,0.077727,0.066981,-0.061251,-0.269633,-0.369456,-0.346828,0.100634,0.180677,-0.152761,-0.342322,-0.663691,-0.542897,-0.041563,-0.104568,0.163341,0.139756,-0.054063,0.015743,-0.508518,-0.486100,-1.044365,-0.496992,0.113961,-0.164476,0.185713,0.146171,-0.079012,0.005056,3810.0,112.0,10.0,20.0,20.0,1.0,1.0,414.643009,0.635838,5.863454,3.420132,1.0,0.198180,4
29,-0.041046,0.128552,0.216366,0.112228,-0.043041,0.166472,0.040723,-0.028979,NDAR_INV00CY2MDM,2_year_follow_up_y_arm_1,-0.014989,0.080711,0.003320,-0.106822,-0.118686,0.027917,-0.054406,0.000477,0.063674,-0.142186,0.016718,-0.137259,-0.025205,-0.077050,-0.100410,-0.090537,-0.093462,-0.216592,-0.061349,-0.124220,-0.107130,-0.153044,-0.111854,-0.087833,-0.063335,-0.181323,0.036538,-0.184872,NaN,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.097978,5


In [31]:
# function
def str2Num(col, istr):
    ### convert string numbers to numeric
    ### istr - specifies the number of strings to convert
    return int(col[-istr:])

if 'siteID' in rs.columns:# if 'siteID' is a column
    # Apply function to siteID col, convert last 2 strings to numeric
    rs['siteID'] = rs['siteID'].apply(str2Num, istr=2)

In [32]:
rs.head()

,NAL_lln,NAR_lln,AmygL_lln,AmygR_lln,HipcL_lln,HipcR_lln,ThalL_lln,ThalR_lln,subID,eventname,NAL_lrn,NAR_lrn,AmygL_lrn,AmygR_lrn,HipcL_lrn,HipcR_lrn,ThalL_lrn,ThalR_lrn,LtOrFrL_lln,LtOrFrR_lln,MedOrFrL_lln,MedOrFrR_lln,rACCL_lln,rACCR_lln,cACCL_lln,cACCR_lln,cACCL_lrn,cACCR_lrn,LtOrFrL_lrn,LtOrFrR_lrn,MedOrFrL_lrn,MedOrFrR_lrn,rACCL_lrn,rACCR_lrn,aInslL_lln,aInslR_lln,aInslL_lrn,aInslR_lrn,famID,age,income,pedu,spedu,gender,race,prpensity,LowEdu1,SingPH1,UnempR1,imgincl_mid_include,Motm,scanID
6,0.012925,0.057350,-0.103007,0.040944,-0.278594,-0.136520,-0.226935,-0.314408,NDAR_INV003RTV85,baseline_year_1_arm_1,-0.451426,-0.238997,-0.327643,-0.223388,-0.471753,-0.240092,-0.138320,-0.261594,-0.071183,-0.014068,0.184538,0.029904,0.053321,0.144795,-0.192958,0.058817,-0.089997,-0.015292,-0.320943,-0.315482,-0.250562,-0.804303,-0.311717,-0.233716,-0.224738,-0.105134,-0.274432,-0.184748,8781.0,131.0,8.0,13.0,13.0,2.0,1.0,466.092707,NaN,NaN,NaN,1.0,0.158333,1
14,0.429318,0.272447,0.354449,0.593416,0.234241,0.289909,0.017087,0.140608,NDAR_INV005V6D2C,baseline_year_1_arm_1,0.142344,0.284221,0.240394,0.450072,0.131028,0.231011,0.043190,0.068502,0.441208,0.173635,0.620966,0.564494,0.540422,0.546842,0.132752,0.293214,-0.080159,0.038292,0.384632,0.078511,0.239707,0.132052,0.406179,0.246562,0.212062,0.198078,0.052116,-0.058588,10210.0,121.0,999.0,6.0,999.0,2.0,3.0,520.488325,4.861931,29.079160,9.991899,1.0,0.352267,2
20,-0.007370,-0.099530,-0.245714,-0.077979,-0.154156,-0.118741,-0.094763,-0.067210,NDAR_INV007W6H7B,baseline_year_1_arm_1,0.024064,-0.018880,-0.231686,-0.010857,-0.064591,-0.160605,-0.088715,-0.187258,-0.216364,-0.361254,0.003707,-0.123096,-0.181924,-0.007465,-0.092264,-0.091089,-0.186848,-0.159265,-0.259863,-0.227784,-0.275979,-0.273947,-0.375694,-0.290470,-0.060283,-0.222956,-0.010167,-0.118855,4722.0,126.0,10.0,19.0,18.0,1.0,1.0,479.185338,3.559711,0.000000,6.254295,1.0,0.086835,3
26,-0.177538,-0.140450,-0.357152,-0.321334,-0.084301,-0.276284,0.178833,0.292628,NDAR_INV00BD7VDC,baseline_year_1_arm_1,0.077727,0.066981,-0.061251,-0.269633,-0.369456,-0.346828,0.100634,0.180677,-0.152761,-0.342322,-0.663691,-0.542897,-0.041563,-0.104568,0.163341,0.139756,-0.054063,0.015743,-0.508518,-0.486100,-1.044365,-0.496992,0.113961,-0.164476,0.185713,0.146171,-0.079012,0.005056,3810.0,112.0,10.0,20.0,20.0,1.0,1.0,414.643009,0.635838,5.863454,3.420132,1.0,0.198180,4
29,-0.041046,0.128552,0.216366,0.112228,-0.043041,0.166472,0.040723,-0.028979,NDAR_INV00CY2MDM,2_year_follow_up_y_arm_1,-0.014989,0.080711,0.003320,-0.106822,-0.118686,0.027917,-0.054406,0.000477,0.063674,-0.142186,0.016718,-0.137259,-0.025205,-0.077050,-0.100410,-0.090537,-0.093462,-0.216592,-0.061349,-0.124220,-0.107130,-0.153044,-0.111854,-0.087833,-0.063335,-0.181323,0.036538,-0.184872,NaN,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.097978,5


### only if not already renamed
### Create  shorter names for correlation variables

In [33]:
corr_cols = [c for c in rs.columns if '_cor_' in c or '_c_' in c]
corr_cols

[]

In [34]:

# FOR resting fmri CORRELATION cols
corr_cols = [c for c in rs.columns if '_cor_' in c or '_c_' in c]
# This wont run if corr_cols is empty
# if corr_cols not empty
if corr_cols:
    # columns for df
    corr_cols_cfa = [c for c in corr_cols if '/' not in c]
    corr_cols_cfa = [c for c in corr_cols_cfa if 'site_id_l' not in c]
    # create shorter variable names for viewing in R
    corr_cols_cfa_sn = ['_'.join(c.split('_')[-3:]) for c in corr_cols_cfa]
    rcols = dict(zip(corr_cols_cfa, corr_cols_cfa_sn))
    rs.rename(columns=rcols, inplace=True)

# Desikan regins and FOR task fmri ACTIVATION cols
# maybe implement programatic names in th future
# tfmri_dsk_cols = 
# if tfmri_dsk_cols:
    # # columns for cfa's
    # tfmri_dsk_cols = [c for c in tfmri_dsk_cols if 'site_id_l' not in c]
    # # create shorter variable names for viewing in R
    # tfmri_dsk_cols_sn = ['_'.join(c.split('_')[-3:]) for c in tfmri_dsk_cols]
    # # zip long names and short names together in Dictionary
    # rcols = dict(zip(tfmri_dsk_cols, tfmri_dsk_cols_sn))
    # rs.rename(columns=rcols, inplace=True)

### Naming key

In [35]:
## CHANGE name of filename ('tfmri_mid_var_name_key.csv') in to_csv function
# convert rename dictionary to dataframe  
names = pd.DataFrame(np.column_stack([list(redict.keys()), list(redict.values())]), columns=['Variable', 'Name'])
# merge with tvars
nkey = tvars.merge(names, how='left', on='Variable')
# get unnamed extra columns
udrop = [c for c in nkey.columns if 'Unnamed' in c]
# drop unnamed extra columns
nkey = nkey.drop(columns = udrop)
# reorder columns for readiblity
nkey = nkey[['Table', 'Variable', 'Name', 'Description']]
# export to csv
nkey.to_csv(fpath + 'rsfmri_var_name_key.csv', index=False)
nkey.head()

,Table,Variable,Name,Description
0,mri_y_tfmr_mid_allvn_aseg,tfmri_ma_aclvn_b_scs_aalh,NAL_lln,Beta weight for MID all anticipation of large ...
1,mri_y_tfmr_mid_allvn_aseg,tfmri_ma_aclvn_b_scs_aarh,NAR_lln,Beta weight for MID all anticipation of large ...
2,mri_y_tfmr_mid_allvn_aseg,tfmri_ma_aclvn_b_scs_aylh,AmygL_lln,Beta weight for MID all anticipation of large ...
3,mri_y_tfmr_mid_allvn_aseg,tfmri_ma_aclvn_b_scs_ayrh,AmygR_lln,Beta weight for MID all anticipation of large ...
4,mri_y_tfmr_mid_allvn_aseg,tfmri_ma_aclvn_b_scs_hpuslh,HipcL_lln,Beta weight for MID all anticipation of large ...


### Save overall df to csv output

In [39]:
rs.shape

(81878, 12)

In [42]:
rs.to_csv(fpath + 'abcd5.1_rtmri_mid_llosVn_lrwdVn_opfc_subc_net_hses.csv', index=False)